In [ ]:
%load_ext autoreload
%autoreload 3

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from splink.duckdb.linker import DuckDBLinker
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_level_library as cll
import splink.duckdb.comparison_template_library as ctl
from splink.comparison import Comparison
import sqlalchemy as sa
import pudl

import ferc1_eia_match

# Setup

Read in FERC1 and EIA inputs (output of candidate set creation set). 

In [ ]:
eia_full = pd.read_pickle("eia_candidates_18_20_k_25.pkl")
ferc_full = pd.read_pickle("ferc_candidates_18_20_k_25.pkl")

In [ ]:
shared_cols = list(set(eia_full.columns) & set(ferc_full.columns))
shared_cols.sort()
shared_cols

In [ ]:
eia_full[shared_cols].isnull().sum().sort_values()

In [ ]:
ferc_full[shared_cols].isnull().sum().sort_values()

TODO: Try experimenting with using more or different columns.

In [ ]:
matching_cols = ["plant_name",
                 "utility_name",
                 "fuel_type_code_pudl",
                 "installation_year",
                 "construction_year",
                 "capacity_mw",
                 # "net_generation_mwh",
                 # "capacity_factor"
                ]
# retain these columns either for blocking or validation, not going to match with these
extra_cols = ["plant_id_pudl", "utility_id_pudl", "report_year", "block_num"]

In [ ]:
ferc_df = ferc_full[matching_cols + extra_cols].reset_index().rename(columns={"record_id_ferc1": "record_id"})
eia_df = eia_full[matching_cols + extra_cols].reset_index().rename(columns={"record_id_eia": "record_id"})

In [ ]:
# eia_df["net_generation_mwh"] = eia_df["net_generation_mwh"].round(2)
# ferc_df["net_generation_mwh"] = ferc_df["net_generation_mwh"].round(2)

In [ ]:
ferc_df["installation_year"] = pd.to_datetime(ferc_df["installation_year"], format="%Y")
ferc_df["construction_year"] = pd.to_datetime(ferc_df["construction_year"], format="%Y")
eia_df["installation_year"] = pd.to_datetime(eia_df["installation_year"], format="%Y")
eia_df["construction_year"] = pd.to_datetime(eia_df["construction_year"], format="%Y")

### Get training data

In [ ]:
pudl_engine = sa.create_engine(pudl.workspace.setup.get_defaults()['pudl_db'])

In [ ]:
train_full = ferc1_eia_match.inputs.InputManager(pudl_engine=pudl_engine, start_report_year="2018", end_report_year="2018").get_training_data()

In [ ]:
train_full

In [ ]:
train_df = train_full[["record_id_ferc1", "record_id_eia"]].rename(columns={"record_id_eia": "record_id_l", "record_id_ferc1": "record_id_r"})
train_df.loc[:, "source_dataset_r"] = "ferc_df"
train_df.loc[:, "source_dataset_l"] = "eia_df"
train_df.loc[:, "clerical_match_score"] = 1 # this column is just a syntax quirk, doesn't mean anything

### Create settings dict and linker

In [ ]:
settings_dict = {"link_type": "link_only",
                 "unique_id_column_name": "record_id",
                 "additional_columns_to_retain": ["plant_id_pudl", "utility_id_pudl"]}

In [ ]:
linker = DuckDBLinker([eia_df, ferc_df], input_table_aliases = ["eia_df", "ferc_df"], settings_dict=settings_dict)

In [ ]:
train_table = linker.register_table(train_df, "training_labels", overwrite=True)

In [ ]:
train_table.as_pandas_dataframe(limit=5)

# Data Exploration

In [ ]:
linker_eia = DuckDBLinker(eia_df)
linker_ferc = DuckDBLinker(ferc_df)

In [ ]:
linker_ferc.missingness_chart()

In [ ]:
linker_eia.missingness_chart()

Columns with higher cardinality are better for matching
- `fuel_type_code_pudl` might not be the best, high skew in that column too

In [ ]:
linker.profile_columns(matching_cols, top_n=10, bottom_n=5)

# Block On `report_year` and `block_num`

`splink` has tools to evaluate more complex blocking rules as well, but since we did blocking a separate step/module, we can just block on `report_year` and `block_num` here.

In [ ]:
blocking_rule = "l.report_year = r.report_year and l.block_num = r.block_num"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule)
print(f"Number of comparisons generated by '{blocking_rule}': {count:,.0f}")

Number of comparisons is a little high for the DuckDB linker when only blocking on report year.

# Define Comparisons

[Comparison Template library](https://moj-analytical-services.github.io/splink/comparison_template_library.html)

In [ ]:
# try without damerau levenshtein
plant_name_comparison = ctl.name_comparison("plant_name", damerau_levenshtein_thresholds=[])
utility_name_comparison = ctl.name_comparison("utility_name", damerau_levenshtein_thresholds=[])

In [ ]:
print(utility_name_comparison.human_readable_description)

In [ ]:
capacity_comparison = {
    "output_column_name": "capacity_mw",
    "comparison_levels": [
        cll.null_level("capacity_mw"),
        cll.percentage_difference_level("capacity_mw", 0.0 + 1e-4),  # could add an exact match level too
        cll.percentage_difference_level("capacity_mw", 0.1 + 1e-4), # need the 1e-4?
        cll.percentage_difference_level("capacity_mw", 0.2 + 1e-4),
        cll.else_level(),
    ],
    "comparison_description": "0% different vs. 10% different vs. 20% different vs. anything else"
}

In [ ]:
print(Comparison(capacity_comparison).human_readable_description)

In [ ]:
net_gen_comparison = {
    "output_column_name": "net_generation_mwh",
    "comparison_levels": [
        cll.null_level("net_generation_mwh"),
        cll.percentage_difference_level("net_generation_mwh", 0.0 + 1e-4),  # could add an exact match level too
        cll.percentage_difference_level("net_generation_mwh", 0.1 + 1e-4), # need the 1e-4?
        cll.percentage_difference_level("net_generation_mwh", 0.2 + 1e-4),
        cll.else_level(),
    ],
    "comparison_description": "0% different vs. 10% different vs. 20% different vs. anything else"
}

capacity_factor_comparison = {
    "output_column_name": "capacity_factor",
    "comparison_levels": [
        cll.null_level("capacity_factor"),
        cll.percentage_difference_level("capacity_factor", 0.0 + 1e-4),  # could add an exact match level too
        cll.percentage_difference_level("capacity_factor", 0.1 + 1e-4), # need the 1e-4?
        cll.percentage_difference_level("capacity_factor", 0.2 + 1e-4),
        cll.else_level(),
    ],
    "comparison_description": "0% different vs. 10% different vs. 20% different vs. anything else"
}

In [ ]:
def get_date_comparison(column_name):
    return ctl.date_comparison(column_name,
                               date_format="%Y",
                               damerau_levenshtein_thresholds=[],
                               datediff_thresholds=[1, 2],
                               datediff_metrics=["year", "year"])

installation_year_comparison = get_date_comparison("installation_year")
construction_year_comparison = get_date_comparison("construction_year")

In [ ]:
print(installation_year_comparison.human_readable_description)

In [ ]:
settings_dict.update({
    "comparisons": [
        plant_name_comparison,
        utility_name_comparison,
        construction_year_comparison,
        installation_year_comparison,
        capacity_comparison,
        cl.exact_match("fuel_type_code_pudl", term_frequency_adjustments=True),
        # net_gen_comparison,
        # capacity_factor_comparison
    ],
    "blocking_rules_to_generate_predictions": [
        blocking_rule
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "probability_two_random_records_match": 1/len(eia_df) # is this correct?
    }
)

In [ ]:
linker.load_settings(settings_dict)

# Estimate Model Parameters

Now that we have specified our linkage model, we need to estimate the probability_two_random_records_match, u, and m parameters.

I think we can use the rationale that for each FERC record there is one EIA matching record. Which means that the probability too records match is 1/n_eia_records.

In [ ]:
# try with a much higher probability of two records matching - this seems wrong
deterministic_rules = [
    "jaro_winkler_similarity(l.plant_name, r.plant_name) >= 0.9 and jaro_winkler_similarity(l.utility_name, r.utility_name) >= 0.9"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


In [ ]:
%%time
linker.estimate_u_using_random_sampling(max_pairs=1e7)

We can estimate m with either training labels or unsupervised, with Expectation Maximization.

In [ ]:
# linker.estimate_m_from_pairwise_labels("training_labels")

In [ ]:
training_blocking_rule_1 = "l.plant_name = r.plant_name"
training_session_1 = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule_1)

In [ ]:
training_blocking_rule_2 = "l.capacity_mw = r.capacity_mw and l.utility_name = r.utility_name"
training_session_2 = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule_2)

In [ ]:
linker.match_weights_chart()

In [ ]:
linker.m_u_parameters_chart()

In [ ]:
# reads like: "a match threshold of 70% will include 94% of records"
linker.unlinkables_chart()

In [ ]:
n = "unsupervised_1"

In [ ]:
settings = linker.save_model_to_json(f"./splink_model_settings/model_settings_{n}.json", overwrite=True)

# Make Predictions

In [ ]:
# df_preds = linker.predict(threshold_match_probability=0.5)
df_preds = linker.predict()

In [ ]:
len(df_preds.as_pandas_dataframe())

In [ ]:
sorted_preds_df = df_preds.as_pandas_dataframe().sort_values(by="match_probability", ascending=False)

In [ ]:
sorted_preds_df.head(3)

In [ ]:
one_to_one_preds = sorted_preds_df.groupby("record_id_r").first()

In [ ]:
cols = [col + "_l" for col in matching_cols]
cols += [col + "_r" for col in matching_cols]
extra_cols = ["plant_id_pudl_l", "plant_id_pudl_r", "utility_id_pudl_l", "utility_id_pudl_r"]
cols.sort()
cols = ["record_id_l", "match_weight", "match_probability"] + cols + extra_cols
one_to_one_preds = one_to_one_preds[cols].reset_index()

In [ ]:
labels_df = train_df.copy()

In [ ]:
n_train_records = len(labels_df)

In [ ]:
# how many FERC records had matches above the match threshold
predicted_train_matches = labels_df.merge(
    one_to_one_preds,
    how="left",
    on=["record_id_r"],
    indicator=True,
    suffixes=("_true", "_pred"))

In [ ]:
# how many FERC train records had matches above the match threshold
predicted_train_matches._merge.value_counts()

In [ ]:
# how many FERC train records were correctly matched
correct_filter = (predicted_train_matches.record_id_l_true == predicted_train_matches.record_id_l_pred)
correct_matches = predicted_train_matches[correct_filter]
len(correct_matches)/n_train_records

In [ ]:
# what do the incorrect matches look like
incorrect_matches = predicted_train_matches[~correct_filter][["record_id_r", "record_id_l_true", "record_id_l_pred", "match_weight", "match_probability"]]
incorrect_matches

Most of the time when records don't match it's because the capacity, installation year, or construction year is a better match for a different record. How to avoid this? Should more columns be included?

What percentage of these incorrect matches didn't have the true record in the blocking set?

In [ ]:
i = 0

In [ ]:
rec_pair = incorrect_matches.iloc[i]
rec_pair

In [ ]:
rec_comparison = sorted_preds_df[(sorted_preds_df.record_id_r == rec_pair.record_id_r) & sorted_preds_df.record_id_l.isin([rec_pair.record_id_l_true, rec_pair.record_id_l_pred])]
rec_comparison

See if there are any notes for that match.

In [ ]:
train_full[train_full.record_id_ferc1.eq(rec_pair.record_id_r)]

In [ ]:
linker.waterfall_chart(rec_comparison.to_dict("records"), filter_nulls=False)

In [ ]:
linker.precision_recall_chart_from_labels_table("training_labels")

Do `utility_id_pudl` and `plant_id_pudl` generally match up?

In [ ]:
consistent_id_df = one_to_one_preds.dropna(subset=["utility_id_pudl_l",
                                                   "utility_id_pudl_r",
                                                   "plant_id_pudl_l",
                                                   "plant_id_pudl_r"
                                                  ])

In [ ]:
(consistent_id_df.plant_id_pudl_l == consistent_id_df.plant_id_pudl_r).value_counts()

In [ ]:
(consistent_id_df.utility_id_pudl_l == consistent_id_df.utility_id_pudl_r).value_counts()